# Desafio técnico Engenheiro de dados Semantix
#### Análise de Web Logs NASA


In [1]:
import os
import sys
import gzip
import shutil
import pyspark
import urllib.request
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import split, regexp_extract, year, month, dayofmonth, udf, isnan, when, count, col, sum

## Criando diretório para armazenar os dados

In [2]:
data_dir = 'data'
if not os.path.exists(data_dir):
    print("Criando diretório '{}'.".format(data_dir))
    os.makedirs(data_dir)

Criando diretório 'data'.


## Realizando Download dos arquivos

In [3]:
def download_file(source_url, destination_file):
    try:
        print("Downloading: {} to {}".format(source_url, destination_file))
        urllib.request.urlretrieve(source_url, destination_file)
    except Exception as e:
        print(e)
        sys.exit(1)

In [4]:
nasa_log_jul95_destination_file = '{}/NASA_access_log_Jul95.gz'.format(data_dir)
nasa_log_aug95_destination_file = '{}/NASA_access_log_Aug95.gz'.format(data_dir)
nasa_log_jul95_source_url = 'ftp://ita.ee.lbl.gov/traces/NASA_access_log_Jul95.gz'
nasa_log_aug95_source_url = 'ftp://ita.ee.lbl.gov/traces/NASA_access_log_Aug95.gz'

In [5]:
download_file(nasa_log_jul95_source_url, nasa_log_jul95_destination_file)
download_file(nasa_log_aug95_source_url, nasa_log_aug95_destination_file)

Downloading: ftp://ita.ee.lbl.gov/traces/NASA_access_log_Jul95.gz to data/NASA_access_log_Jul95.gz
Downloading: ftp://ita.ee.lbl.gov/traces/NASA_access_log_Aug95.gz to data/NASA_access_log_Aug95.gz


## Descompactando Arquivos

In [6]:
nasa_log_jul95_destination_raw_file = '{}/nasa_access_log_jul95'.format(data_dir)
nasa_log_aug95_destination_raw_file = '{}/nasa_access_log_aug95'.format(data_dir)

In [7]:
def decompress_file(source_compressed_file, destination_raw_file):
    print('Decompressing file {} to {}'.format(source_compressed_file, destination_raw_file))
    with gzip.open(source_compressed_file, 'rb') as c:
        with open(destination_raw_file, 'wb') as f:
            shutil.copyfileobj(c, f)

In [8]:
decompress_file(nasa_log_jul95_destination_file, nasa_log_jul95_destination_raw_file)
decompress_file(nasa_log_aug95_destination_file, nasa_log_aug95_destination_raw_file)

Decompressing file data/NASA_access_log_Jul95.gz to data/nasa_access_log_jul95
Decompressing file data/NASA_access_log_Aug95.gz to data/nasa_access_log_aug95


### Realizando a leitura dos arquivos

In [9]:
spark = SparkSession.builder \
        .master("local") \
        .appName("NASA Web Log Analysis") \
        .getOrCreate()

In [10]:
df_nasa_log_jul95 = spark.read.text(nasa_log_jul95_destination_raw_file)
df_nasa_log_jul95.printSchema()

df_jul_count = df_nasa_log_jul95.count()
print('Número de registros em {} : {}'.format(nasa_log_jul95_destination_raw_file, df_jul_count))

root
 |-- value: string (nullable = true)

Número de registros em data/nasa_access_log_jul95 : 1891715


In [11]:
df_nasa_log_aug95 = spark.read.text(nasa_log_aug95_destination_raw_file)
df_nasa_log_aug95.printSchema()
df_nasa_log_aug95.count()

df_aug_count = df_nasa_log_aug95.count()
print('Número de registros em {} : {}'.format(nasa_log_aug95_destination_raw_file, df_aug_count))

root
 |-- value: string (nullable = true)

Número de registros em data/nasa_access_log_aug95 : 1569898


In [12]:
total_registros = df_jul_count + df_aug_count
print('Número de registros importados: {}'.format(total_registros))

Número de registros importados: 3461613


### Concatenando dataframes

In [13]:
df_raw_nasa_log = df_nasa_log_aug95.union(df_nasa_log_jul95)
df_raw_nasa_log.printSchema()
df_raw_nasa_log.count()

root
 |-- value: string (nullable = true)



3461613

In [14]:
df_raw_nasa_log.show(n=5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                     |
+--------------------------------------------------------------------------------------------------------------------------+
|in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839|
|uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0                                                   |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0                          |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0                        |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0                           |


### Realizando o Parse dos logs do servidor web

In [15]:
df_nasa_log = df_raw_nasa_log.select(regexp_extract('value', r'^([^\s]+\s)', 1).alias('host'),
                          regexp_extract('value', r'^.*\[(\d\d/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]', 1).alias('timestamp'),
                          regexp_extract('value', r'^.*"\w+\s+([^\s]+)\s+HTTP.*"', 1).alias('path'),
                          regexp_extract('value', r'^.*"\s+([^\s]+)', 1).cast('integer').alias('status'),
                          regexp_extract('value', r'^.*\s+(\d+)$', 1).cast('integer').alias('content_size'))
df_nasa_log.show(n=5, truncate=False)
df_nasa_log.cache()
df_nasa_log.count()

+------------------+--------------------------+-----------------------------------------------+------+------------+
|host              |timestamp                 |path                                           |status|content_size|
+------------------+--------------------------+-----------------------------------------------+------+------------+
|in24.inetnebr.com |01/Aug/1995:00:00:01 -0400|/shuttle/missions/sts-68/news/sts-68-mcc-05.txt|200   |1839        |
|uplherc.upl.com   |01/Aug/1995:00:00:07 -0400|/                                              |304   |0           |
|uplherc.upl.com   |01/Aug/1995:00:00:08 -0400|/images/ksclogo-medium.gif                     |304   |0           |
|uplherc.upl.com   |01/Aug/1995:00:00:08 -0400|/images/MOSAIC-logosmall.gif                   |304   |0           |
|uplherc.upl.com   |01/Aug/1995:00:00:08 -0400|/images/USA-logosmall.gif                      |304   |0           |
+------------------+--------------------------+-------------------------

3461613

## Checagem de dados

### Verificando colunas com registros nulos

In [16]:
df_nasa_log.select([count(when(col(c).isNull(), c)).alias(c) for c in df_nasa_log.columns]).show()

+----+---------+----+------+------------+
|host|timestamp|path|status|content_size|
+----+---------+----+------+------------+
|   0|        0|   0|     1|       33905|
+----+---------+----+------+------------+



### Corrigindo 'content-size' vazio

In [17]:
df_empty_content_size = df_nasa_log.filter(df_nasa_log.content_size.isNull())
df_empty_content_size.show()

+--------------------+--------------------+--------------------+------+------------+
|                host|           timestamp|                path|status|content_size|
+--------------------+--------------------+--------------------+------+------------+
|        gw1.att.com |01/Aug/1995:00:03...|/shuttle/missions...|   302|        null|
|js002.cc.utsunomi...|01/Aug/1995:00:07...|/shuttle/resource...|   404|        null|
|    tia1.eskimo.com |01/Aug/1995:00:28...|/pub/winvn/releas...|   404|        null|
|itws.info.eng.nii...|01/Aug/1995:00:38...|/ksc.html/facts/a...|   403|        null|
|grimnet23.idirect...|01/Aug/1995:00:50...|/www/software/win...|   404|        null|
|miriworld.its.uni...|01/Aug/1995:01:04...|/history/history.htm|   404|        null|
|      ras38.srv.net |01/Aug/1995:01:05...|/elv/DELTA/uncons...|   404|        null|
| cs1-06.leh.ptd.net |01/Aug/1995:01:17...|                    |   404|        null|
|www-b2.proxy.aol....|01/Aug/1995:01:22...|  /shuttle/countdown| 

In [18]:
df_empty_content_size.groupBy('status').count().show()

+------+-----+
|status|count|
+------+-----+
|   501|   41|
|  null|    1|
|   400|   15|
|   403|  225|
|   404|20900|
|   200|  161|
|   302|12562|
+------+-----+



### Substituindo 'content-size' nulo para '0'

In [19]:
df_nasa_log = df_nasa_log.na.fill({'content_size': 0})

## Transformação da coluna 'timestamp'

In [20]:
months = {
    'Jan':1,
    'Feb':2,
    'Mar':3,
    'Apr':4,
    'May':5,
    'Jun':6,
    'Jul':7,
    'Aug':8,
    'Sep':9,
    'Oct':10,
    'Nov':11,
    'Dec':12
}

def parse_timestamp(s):
    return "{0:04d}-{1:02d}-{2:02d} {3:02d}:{4:02d}:{5:02d}".format(
      int(s[7:11]),
      months[s[3:6]],
      int(s[0:2]),
      int(s[12:14]),
      int(s[15:17]),
      int(s[18:20])
    )

u_parse_timestamp = udf(parse_timestamp, StringType())


df_nasa_log = df_nasa_log.select('*', u_parse_timestamp(df_nasa_log['timestamp']).cast('timestamp') \
                                 .alias('time')) \
                                 .drop('timestamp')

## Encontrando o número de hosts unicos

In [21]:
distinct_hosts = df_nasa_log.select('host') \
                .distinct() \
                .count()
print("Número de Hosts Unicos: {}".format(distinct_hosts))

Número de Hosts Unicos: 137979


## Encontrando número de respostas 404 (Not Found)

In [22]:
not_found = df_nasa_log.filter('status = 404').count()
print("Número de Respostas 404: {}".format(not_found))

Número de Respostas 404: 20901


## Encontrando TOP 5 URLs não encontradas

In [23]:
urls = df_nasa_log.filter('status = 404') \
        .groupBy('path') \
        .count() \
        .sort("count", ascending=False)
urls.show(5, truncate=False)

+--------------------------------------------+-----+
|path                                        |count|
+--------------------------------------------+-----+
|/pub/winvn/readme.txt                       |2004 |
|/pub/winvn/release.txt                      |1732 |
|/shuttle/missions/STS-69/mission-STS-69.html|682  |
|/shuttle/missions/sts-68/ksc-upclose.gif    |426  |
|/history/apollo/a-001/a-001-patch-small.gif |384  |
+--------------------------------------------+-----+
only showing top 5 rows



## Encontrando número de Erros 404 por dia

In [24]:
not_found_per_day = df_nasa_log.select(month('time').alias('month'), \
                                       dayofmonth('time').alias('day')) \
                    .filter('status = 404') \
                    .groupBy('month', 'day') \
                    .count() \
                    .sort("count", ascending=False)
not_found_per_day.show(truncate=False)


+-----+---+-----+
|month|day|count|
+-----+---+-----+
|7    |6  |640  |
|7    |19 |639  |
|8    |30 |571  |
|7    |7  |570  |
|8    |7  |537  |
|7    |13 |532  |
|8    |31 |526  |
|7    |5  |497  |
|7    |3  |474  |
|7    |11 |471  |
|7    |12 |471  |
|7    |18 |465  |
|7    |25 |461  |
|7    |20 |428  |
|8    |24 |420  |
|8    |29 |420  |
|8    |25 |415  |
|7    |14 |413  |
|8    |28 |410  |
|7    |17 |406  |
+-----+---+-----+
only showing top 20 rows



## Encontrando o número de bytes trafegados

In [26]:
response_bytes_size = df_nasa_log.agg(sum("content_size")).collect()[0][0]
print('Total de bytes trafegados: {}'.format(response_bytes_size))

Total de bytes trafegados: 65524314915
